## Cross Validation

    Cross Validation is a very important technique that is used widely by data scientists. The problem with machine learning models is that you won’t get to know how well a model performs until you test its performance on an independent data set (the data set which was not used for training the machine learning model).Cross Validation comes to the rescue here and helps you estimate the performance of your model during the training time.One type of cross validation is the K-Fold Cross Validation, which we will discuss in this tutorial.

### K-Fold Cross Validation

    K--fold cross validation works by breaking your training data into K equal-sized “folds.” It iterates through each fold, treating that fold as holdout data, training a model on all the other K-1 folds, and evaluating the model’s performance on the one holdout fold. This results in having K different models, each with an out of sample model accuracy score on a different holdout set. The average of these K models’ out-of-sample scores is the model’s cross-validation score.
    
    As there is never enough data to train a model, removing a part of it for validation poses a problem of underfitting. By reducing the training data, we risk losing important patterns/ trends in data set, which in turn increases error induced by bias. So, what we require is a method that provides ample data for training the model and also leaves ample data for validation. K Fold cross validation does exactly that.

    It can be viewed as repeated holdout and we simply average scores after K different holdouts. Every data point gets to be in a validation set exactly once, and gets to be in a training set k-1times. This significantly reduces underfitting as we are using most of the data for fitting, and also significantly reduces overfitting as most of the data is also being used in validation set.
    
    
   ![alt text](kfold.png "Title")
   
    This method is a good choice when we have a minimum amount of data and we get sufficiently big difference in quality or different optimal parameters between folds. As a general rule, we choose k=5 or k=10, as these values have been shown empirically to yield test error estimates that suffer neither from excessively high bias nor high variance.
    

### Implementation

#### Importing data and libraries

    I’ll use the Iris dataset and a random forest classifier for this example. The dataset, model, and cross validation function can all be imported from Scikit-Learn.

In [13]:
# import random search, random forest, iris data, and distributions
from sklearn.model_selection import cross_validate
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier

# get iris data
iris = datasets.load_iris()
X = iris.data
y = iris.target


#### 1. Train and Evaluate a Model Using K-Fold Cross Validation

    Here I initialize a random forest classifier and feed it to sklearn’s cross_validate function. This function receives a model, its training data, the array or dataframe column of target values, and the number of folds for it to cross validate over (the number of models it will train).

    The cross validation object stores the out of sample accuracy of each of its trained models. Taking the average of these K out-of-sample scores gives you the model’s cross-validation accuracy, a low-variance estimate of how the model will perform on unseen data.

In [14]:
model = DecisionTreeClassifier()
cv = cross_validate(model, X, y, cv=5)
print("Accuracy on each fold :",cv['test_score'])
print("Mean accuracy : ",cv['test_score'].mean())


Accuracy on each fold : [0.96666667 0.96666667 0.9        0.96666667 1.        ]
Mean accuracy :  0.9600000000000002


    Cross validation should be used whenever assessing the quality of a model’s predictions. It is also baked into common parameter tuning methods such as grid search cross validation.

#### 2. Train and Evaluate a Model Using K-Fold Cross Validation


In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

# set to 10 folds
skf = StratifiedKFold(n_splits=10)

i=0
# blank lists to store predicted values and actual values
predicted_y = []
expected_y = []
# partition data
for train_index, test_index in skf.split(X, y):
    # specific ".loc" syntax for working with dataframes
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # create and fit classifier
    classifier = DecisionTreeClassifier()
    
    ## Training on first selected features
    classifier.fit(x_train, y_train)
    
    # store result from classification
    predicted_y.extend(classifier.predict(x_test))

    # store expected result for this specific fold
    expected_y.extend(y_test)

    # save and print accuracy
    accuracy = metrics.accuracy_score(expected_y, predicted_y)
    print("Fold" ,i+1," Accuracy: " + accuracy.__str__())
    i=i+1


Fold 1  Accuracy: 1.0
Fold 2  Accuracy: 0.9666666666666667
Fold 3  Accuracy: 0.9777777777777777
Fold 4  Accuracy: 0.9666666666666667
Fold 5  Accuracy: 0.96
Fold 6  Accuracy: 0.9444444444444444
Fold 7  Accuracy: 0.9428571428571428
Fold 8  Accuracy: 0.95
Fold 9  Accuracy: 0.9555555555555556
Fold 10  Accuracy: 0.96


## Tune Hyperparameters with Grid Search

#### Background

    Grid search is a common method for tuning a model’s hyperparameters. The grid search algorithm is simple: you feed it a set of hyperparameters and the values you want to test for each hyperparameter, and then run an exhaustive search over all possible combinations of these values, training one model for each set of values. The algorithm then compares the scores of each model it trains and keeps the best one. A common extension of grid search is to use it alongside cross validation, training on several different folds for each hyperparameter combination to have a more reliable estimate of each model’s performance out of sample

#### Importing data and libraries

    I’ll use the Iris dataset because it’s easy to import from Scikit-Learn. The model we tune using grid search will be a random forest classifier.

In [1]:
# import random search, random forest, and iris data
from sklearn.model_selection import GridSearchCV
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
# get iris data
iris = datasets.load_iris()
X = iris.data
y = iris.target


#### Set Up a Grid of Hyperparameter Values

    I’ll tune three hyperparameters: n_estimators, max_features, and min_samples_split. For each parameter I’ll need to form a hypothesis about which values I think will work in the model; you need to explicitly state each value you want grid search to test.

    The hyperparameter values are set up as a dictionary so they can be passed to GridSearchCV.

In [8]:
model_params = {
    
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 5, 10],
    'min_samples_split': [2, 4, 6]
}


#### Define and Train the Model with Grid Search

    The most important arguments to pass to GridSearchCV are the model you’re training, the dictionary of parameter values you’re testing, and the number of folds for it to cross validate over.

    The grid search meta-estimator runs an exhaustive search over all possible combinations of the hyperparameter values you pass to it. The number of cross validation folds you choose determines how many times it will train each model on a different subset of data in order to assess model quality.

    The result of training the grid search meta-estimator will be the best model that it finds across all candidate models.

In [10]:
# create random forest classifier model
rf_model = DecisionTreeClassifier()

# set up grid search meta-estimator
clf = GridSearchCV(rf_model, model_params, cv=5)

# train the grid search meta-estimator to find the best model
model = clf.fit(X, y)

# print winning set of hyperparameters
from pprint import pprint
pprint(model.best_estimator_.get_params())


{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}


#### Generate Predictions Using the Best Model

    Now that you have a tuned model, there’s no additional work needed to generate predictions from the best model that grid search found. Simply call .predict() on the meta-estimator and you’re good to go.

In [11]:
# generate predictions using the best-performing model
predictions = model.predict(X)
print(predictions)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


#### Conclusion

    Grid search is a useful algorithm for tuning model parameters and improving overall model quality. Grid search will only explore the hyperparameter values you tell it to use, while random search is able to draw hyperparameter values from continuous distributions, allowing it to sample the parameter space more fully and efficiently.

    Grid search is also an epensive algorithm. You’ll want to choose your parameter values carefully. For each additional value you add to your grid search for one hyperparameter, the algorithm will train it against all other possible values of all the other parameters you’re tuning. Paired with 5-fold cross validation, it will do this five times. This can quickly spiral out of control into a model that never finishes training.

## TO-DO LIST

1) Hyper-parameter tuning is one of the very important step in model training, try to hypertune parameters of any other classifier/ Regressor.

2) There are other methods then Grid search e.g randomizedsearchcv. Try to use that for tuning.

3) Try the k-fold on a randomly imbalance dataset, and see how model performs on each fold.